# Watershed Summaries

1. read in all watersheds feature classes
2. create a table with the NHDPlusID/catID of the watershed name, region, and watershed area
3. merge original point feature classes (inside and outside bb)
4. do a spatial join with merged catchments by region to get catID and region on the points dataset
5. join the watershed area to the points dataset using the catID

In [ ]:
# steps 1 and 2

import arcpy
import os
import pandas as pd

# regions = ["Copper_River"]
regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]

for region in regions:
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb\\Watersheds"
    wtds = arcpy.ListFeatureClasses()
    print(wtds)
    print(len(wtds))
    wtdList = []

    for wtd in wtds:
        wtdName = wtd[4:20]
        print("Starting wtd: " + wtdName)
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        arcpy.AddField_management(wtdPath, "Area_km2", "DOUBLE")
        expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
        arcpy.CalculateField_management(wtdPath, "Area_km2", expression1, "PYTHON", )
        wtdArea = [row[0] for row in arcpy.da.SearchCursor(wtdPath, ['Area_km2'])]
        print("wtdName: " + str(wtdArea))
        wtdList.append({'Region': region, 'Name': wtdName, 'Area_km2': wtdArea})

wtdDf = pd.DataFrame(wtdList)
print(wtdDf)


In [ ]:
# step 3
import arcpy


gdb = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb"
arcpy.env.workspace = gdb
bb_pts = gdb + "\\bb_md_verified_DM"
other_pts = gdb + "\\sites_outside_bb_verified_DM"

output = "akssf_pts_verified"
arcpy.Merge_management([bb_pts, other_pts], output)


In [ ]:
# step 4

import arcpy
import os
import numpy
import pandas as pd

arcpy.env.overwriteOutput = True
regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]
# regions = ["Kodiak"]
points = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\akssf_pts_verified"
sites_lst = []

for region in regions:
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb"
    cats = os.path.join(arcpy.env.workspace, "cats_merge")

    arcpy.SpatialJoin_analysis(points, cats, "sites_all_sj")
    #follow up with clip so just points in that region
    arcpy.Clip_analysis("sites_all_sj", cats, "sites_sj")
    arcpy.DeleteFeatures_management("sites_all_sj")
    sites_sj = os.path.join(arcpy.env.workspace, "sites_sj")
    sites_lst.append(os.path.join(arcpy.env.workspace, "sites_sj"))


#merge all sites_sj into one point file with all the catIDs.
all_sites_out = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\all_sites_wcatid"
arcpy.Merge_management(sites_lst, all_sites_out)


In [ ]:
# step 5

import arcpy

#save to dataframe or csv
sitesDf = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(all_sites_out))

#merge sitesDf with wtdDf to get watershed area linked to siteID


